# Group M - Solar Potential ML
## Members:
### 21030062 - Eimantas Miliauskis
### - Harvey
### - Yasine

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import accuracy_score, precision_score, classification_report

In [ ]:
readfile = pd.read_csv("solar-potential.csv")